In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [22]:
import json
with open("service1906_1506.json") as of:
    data = json.load(of)

In [23]:
computes = [c for c in data.keys() if c!="timespan"]
variables = [v for v in data[computes[0]] if v!='index' and v!='arrJob_scheduling']

In [24]:
#Check empty array
def getEmptyArr(data, c):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            print('c=', c)
            print('v=', v)
for c in computes:
    getEmptyArr(data, c)

c= compute-1-26
v= arrTemperature


In [25]:
def addTarget(cDf, predictedVar, predictedStep, target):
    cDf[target] = cDf[predictedVar].shift(-predictedStep)
    cDf.dropna(inplace=True)
    return cDf

In [26]:
def getComputeDf(data, c, predictedVar, predictedStep, target):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            return None
        else:
            for i in range(len(vArr[0])):
                cDf[v+str(i)] = vArr[:, i]
    cDf['timespan'] = pd.to_datetime(cDf['timespan'])
    return addTarget(cDf, predictedVar, predictedStep, target)

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import FileLink
import codecs, json

In [28]:
def exportNPArrayToJson(a, fileName):
    b = a.tolist()
    json.dump(b, codecs.open(fileName, 'w', encoding='utf-8'))

In [38]:
def splitTrainTest(predictedVar):
    target = predictedVar + "_target"
    predictedSteps = 4
    df = pd.concat([x for x in [getComputeDf(data, c, predictedVar, predictedSteps, target) for c in computes] if type(x)!="NoneType"])
    
    df = df.reset_index().drop('index', axis=1)
    features = [x for x in df.columns if x not in ['compute', 'timespan', target]]
        
    X_dfs = []
    y = []
    numberOfSequences = 1
    sequenceSteps = 100
    # generate training data.
    for compute in computes:
        cDf = df[df['compute']==compute]
        if(len(cDf) > sequenceSteps):
            randSteps = np.random.randint(0, len(cDf)-sequenceSteps, numberOfSequences)
            for randStep in randSteps:
                X_dfs.append(cDf.iloc[randStep:randStep+sequenceSteps])
                y.append(X_dfs[-1][target].values[-1])

    X_train_dfs, X_test_dfs, y_train, y_test = train_test_split(X_dfs, y, test_size=0.33)
            
    #Scale data

    # combine the training data to create a scaler
    train_dfs = pd.concat(X_train_dfs)

    scaler = StandardScaler().fit(train_dfs[features].values)
    
    X_train = np.array([scaler.transform(item[features].values) for item in X_train_dfs])
    X_test = np.array([scaler.transform(item[features].values) for item in X_test_dfs])
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    exportNPArrayToJson(X_train, 'allData100/'+target+'_X_train_HPCC_1_100.json')
    exportNPArrayToJson(X_test, 'allData100/'+target+'_X_test_HPCC_1_100.json')
    exportNPArrayToJson(y_train, 'allData100/'+target+'_y_train_HPCC_1_100.json')
    exportNPArrayToJson(y_test, 'allData100/'+target+'_y_test_HPCC_1_100.json')
    
    FileLink('allData100/'+target+'_X_train_HPCC_1_100.json')
    FileLink('allData100/'+target+'_X_test_HPCC_1_100.json')
    FileLink('allData100/'+target+'_y_train_HPCC_1_100.json')
    FileLink('allData100/'+target+'_y_test_HPCC_1_100.json')
    
    return (X_train, X_test, y_train, y_test)

In [39]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten

# from keras import backend as K
# K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=36, inter_op_parallelism_threads=36)))


def createModel(l1Nodes, l2Nodes, d1Nodes, d2Nodes, inputShape):
    # input layer
    lstm1 = LSTM(l1Nodes, input_shape=inputShape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do1 = Dropout(0.2)
    
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do2 = Dropout(0.2)
    
    flatten = Flatten()
    
    dense1 = Dense(d1Nodes, activation='relu')
    do3 = Dropout(0.2)
    
    dense2 = Dense(d2Nodes, activation='relu')
    do4 = Dropout(0.2)
    
    # output layer
    outL = Dense(1, activation='relu')
    # combine the layers
#     layers = [lstm1, do1, lstm2, do2, dense1, do3, dense2, do4, outL]
    layers = [lstm1, lstm2, flatten,  dense1, dense2, outL]
    # create the model
    model = Sequential(layers)
    model.compile(optimizer='adam', loss='mse')
    return model

In [40]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
# ten fold
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3, shuffle=True)
from keras.models import load_model
from datetime import datetime

msescores = []

allVariables = ['arrTemperature0', 'arrTemperature1', 'arrTemperature2', 'arrCPU_load0', 'arrMemory_usage0', 'arrFans_health0', 'arrFans_health1', 'arrFans_health2', 'arrFans_health3', 'arrPower_usage0']

start_time = datetime.now()
print(start_time)
for feature in allVariables:
    print(feature)
    X_train, X_test, y_train, Y_test = splitTrainTest(feature)

    counter= 0
    for trainIdx, testIdx in kfold.split(X_train, y_train):
        counter = counter + 1
        # create callbacks
        model_path = feature+'_best_model_fold'+str(counter)+"_"+datetime.now().strftime("%H_%M_%S")+'.h5'
        mc = ModelCheckpoint(model_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        es = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1)
        # create model
        model = createModel(8, 8, 8, 4, (X_train.shape[1], X_train.shape[2]))
        model.fit(X_train[trainIdx], y_train[trainIdx], validation_data=(X_train[testIdx], y_train[testIdx]), batch_size=32, epochs=40, callbacks=[mc, es])
        # Done load the best model of this fold
        saved_model = load_model(model_path)
        msescores.append({'path': model_path, 'mse': saved_model.evaluate(X_train[testIdx], y_train[testIdx])})
        
end_time = datetime.now()

2019-10-30 19:47:59.987027
arrTemperature0
Train on 208 samples, validate on 104 samples
Epoch 1/40
208/208 [==============================] - 1s 7ms/step - loss: 3680.9407 - val_loss: 3396.0529

Epoch 00001: val_loss improved from inf to 3396.05287, saving model to arrTemperature0_best_model_fold1_19_48_24.h5
Epoch 2/40
208/208 [==============================] - 1s 3ms/step - loss: 2567.4265 - val_loss: 1336.7699

Epoch 00002: val_loss improved from 3396.05287 to 1336.76990, saving model to arrTemperature0_best_model_fold1_19_48_24.h5
Epoch 3/40
208/208 [==============================] - 1s 3ms/step - loss: 857.1462 - val_loss: 770.5220

Epoch 00003: val_loss improved from 1336.76990 to 770.52204, saving model to arrTemperature0_best_model_fold1_19_48_24.h5
Epoch 4/40
208/208 [==============================] - 1s 3ms/step - loss: 624.6047 - val_loss: 453.6032

Epoch 00004: val_loss improved from 770.52204 to 453.60324, saving model to arrTemperature0_best_model_fold1_19_48_24.h5
Epoch

208/208 [==============================] - 1s 3ms/step - loss: 106.6702 - val_loss: 127.9030

Epoch 00021: val_loss did not improve from 122.54612
Epoch 22/40
208/208 [==============================] - 1s 3ms/step - loss: 101.4799 - val_loss: 111.4110

Epoch 00022: val_loss improved from 122.54612 to 111.41098, saving model to arrTemperature0_best_model_fold2_19_48_37.h5
Epoch 23/40
208/208 [==============================] - 1s 3ms/step - loss: 89.3120 - val_loss: 108.4813

Epoch 00023: val_loss improved from 111.41098 to 108.48127, saving model to arrTemperature0_best_model_fold2_19_48_37.h5
Epoch 24/40
208/208 [==============================] - 1s 2ms/step - loss: 86.3061 - val_loss: 111.3917

Epoch 00024: val_loss did not improve from 108.48127
Epoch 25/40
208/208 [==============================] - 1s 3ms/step - loss: 84.2337 - val_loss: 107.2030

Epoch 00025: val_loss improved from 108.48127 to 107.20301, saving model to arrTemperature0_best_model_fold2_19_48_37.h5
Epoch 26/40
208/

208/208 [==============================] - 0s 2ms/step - loss: 119.4114 - val_loss: 74.2534

Epoch 00009: val_loss improved from 264.84625 to 74.25335, saving model to arrTemperature1_best_model_fold1_19_49_40.h5
Epoch 10/40
208/208 [==============================] - 0s 2ms/step - loss: 75.5871 - val_loss: 71.3676

Epoch 00010: val_loss improved from 74.25335 to 71.36762, saving model to arrTemperature1_best_model_fold1_19_49_40.h5
Epoch 11/40
208/208 [==============================] - 0s 2ms/step - loss: 70.1370 - val_loss: 61.3140

Epoch 00011: val_loss improved from 71.36762 to 61.31402, saving model to arrTemperature1_best_model_fold1_19_49_40.h5
Epoch 12/40
208/208 [==============================] - 1s 2ms/step - loss: 60.7663 - val_loss: 55.2017

Epoch 00012: val_loss improved from 61.31402 to 55.20174, saving model to arrTemperature1_best_model_fold1_19_49_40.h5
Epoch 13/40
208/208 [==============================] - 0s 2ms/step - loss: 56.6606 - val_loss: 52.9426

Epoch 00013: v

Epoch 20/40
208/208 [==============================] - 1s 3ms/step - loss: 39.8373 - val_loss: 48.4302

Epoch 00020: val_loss did not improve from 48.06114
Epoch 21/40
208/208 [==============================] - 1s 3ms/step - loss: 37.1545 - val_loss: 48.3251

Epoch 00021: val_loss did not improve from 48.06114
Epoch 22/40
208/208 [==============================] - 1s 2ms/step - loss: 35.3512 - val_loss: 48.5566

Epoch 00022: val_loss did not improve from 48.06114
Epoch 00022: early stopping
104/104 [==============================] - 0s 2ms/step
Train on 208 samples, validate on 104 samples
Epoch 1/40
208/208 [==============================] - 1s 6ms/step - loss: 2080.4337 - val_loss: 1616.3889

Epoch 00001: val_loss improved from inf to 1616.38892, saving model to arrTemperature1_best_model_fold3_19_50_15.h5
Epoch 2/40
208/208 [==============================] - 0s 2ms/step - loss: 1120.6648 - val_loss: 511.7565

Epoch 00002: val_loss improved from 1616.38892 to 511.75647, saving model 

208/208 [==============================] - 0s 2ms/step - loss: 10.0447 - val_loss: 14.1680

Epoch 00015: val_loss improved from 14.89772 to 14.16802, saving model to arrTemperature2_best_model_fold1_19_50_58.h5
Epoch 16/40
208/208 [==============================] - 0s 2ms/step - loss: 9.7788 - val_loss: 14.1086

Epoch 00016: val_loss improved from 14.16802 to 14.10857, saving model to arrTemperature2_best_model_fold1_19_50_58.h5
Epoch 17/40
208/208 [==============================] - 0s 2ms/step - loss: 9.2847 - val_loss: 13.2702

Epoch 00017: val_loss improved from 14.10857 to 13.27022, saving model to arrTemperature2_best_model_fold1_19_50_58.h5
Epoch 18/40
208/208 [==============================] - 0s 2ms/step - loss: 8.6239 - val_loss: 13.1696

Epoch 00018: val_loss improved from 13.27022 to 13.16961, saving model to arrTemperature2_best_model_fold1_19_50_58.h5
Epoch 19/40
208/208 [==============================] - 0s 2ms/step - loss: 8.3151 - val_loss: 12.6153

Epoch 00019: val_los

...

104/104 [==============================] - 0s 2ms/step
Train on 208 samples, validate on 104 samples
Epoch 1/40
208/208 [==============================] - 1s 7ms/step - loss: 83525360.6154 - val_loss: 86261525.5385

Epoch 00001: val_loss improved from inf to 86261525.53846, saving model to arrFans_health3_best_model_fold2_20_02_03.h5
Epoch 2/40
208/208 [==============================] - 1s 2ms/step - loss: 83198233.2308 - val_loss: 85523966.7692

Epoch 00002: val_loss improved from 86261525.53846 to 85523966.76923, saving model to arrFans_health3_best_model_fold2_20_02_03.h5
Epoch 3/40
208/208 [==============================] - 1s 2ms/step - loss: 82258719.3846 - val_loss: 84154868.3077

Epoch 00003: val_loss improved from 85523966.76923 to 84154868.30769, saving model to arrFans_health3_best_model_fold2_20_02_03.h5
Epoch 4/40
208/208 [==============================] - 1s 2ms/step - loss: 80870872.6154 - val_loss: 82073481.2308

Epoch 00004: val_loss improved from 84154868.30769 to 820

208/208 [==============================] - 1s 4ms/step - loss: 1589455.7788 - val_loss: 2170457.6346

Epoch 00034: val_loss improved from 2217875.34615 to 2170457.63462, saving model to arrFans_health3_best_model_fold2_20_02_03.h5
Epoch 35/40
208/208 [==============================] - 1s 4ms/step - loss: 1568518.6731 - val_loss: 2141974.9038

Epoch 00035: val_loss improved from 2170457.63462 to 2141974.90385, saving model to arrFans_health3_best_model_fold2_20_02_03.h5
Epoch 36/40
208/208 [==============================] - 1s 3ms/step - loss: 1552155.4519 - val_loss: 2125423.4231

Epoch 00036: val_loss improved from 2141974.90385 to 2125423.42308, saving model to arrFans_health3_best_model_fold2_20_02_03.h5
Epoch 37/40
208/208 [==============================] - 1s 3ms/step - loss: 1545886.6442 - val_loss: 2110056.6538

Epoch 00037: val_loss improved from 2125423.42308 to 2110056.65385, saving model to arrFans_health3_best_model_fold2_20_02_03.h5
Epoch 38/40
208/208 [===================

208/208 [==============================] - 0s 2ms/step - loss: 49435671.3846 - val_loss: 45940626.1538

Epoch 00027: val_loss improved from 48776081.23077 to 45940626.15385, saving model to arrFans_health3_best_model_fold3_20_02_38.h5
Epoch 28/40
208/208 [==============================] - 0s 2ms/step - loss: 46529856.6154 - val_loss: 43036860.3077

Epoch 00028: val_loss improved from 45940626.15385 to 43036860.30769, saving model to arrFans_health3_best_model_fold3_20_02_38.h5
Epoch 29/40
208/208 [==============================] - 1s 2ms/step - loss: 43541448.0000 - val_loss: 40105494.7692

Epoch 00029: val_loss improved from 43036860.30769 to 40105494.76923, saving model to arrFans_health3_best_model_fold3_20_02_38.h5
Epoch 30/40
208/208 [==============================] - 1s 2ms/step - loss: 40547172.0000 - val_loss: 37156942.3077

Epoch 00030: val_loss improved from 40105494.76923 to 37156942.30769, saving model to arrFans_health3_best_model_fold3_20_02_38.h5
Epoch 31/40
208/208 [===


Epoch 00022: val_loss improved from 152.29038 to 150.77945, saving model to arrPower_usage0_best_model_fold1_20_03_45.h5
Epoch 23/40
208/208 [==============================] - 1s 3ms/step - loss: 57.2659 - val_loss: 153.3722

Epoch 00023: val_loss did not improve from 150.77945
Epoch 24/40
208/208 [==============================] - 1s 2ms/step - loss: 55.2009 - val_loss: 150.1275

Epoch 00024: val_loss improved from 150.77945 to 150.12746, saving model to arrPower_usage0_best_model_fold1_20_03_45.h5
Epoch 25/40
208/208 [==============================] - 1s 2ms/step - loss: 59.4430 - val_loss: 171.2235

Epoch 00025: val_loss did not improve from 150.12746
Epoch 26/40
208/208 [==============================] - 1s 2ms/step - loss: 59.4618 - val_loss: 146.4595

Epoch 00026: val_loss improved from 150.12746 to 146.45950, saving model to arrPower_usage0_best_model_fold1_20_03_45.h5
Epoch 27/40
208/208 [==============================] - 1s 2ms/step - loss: 56.4133 - val_loss: 152.8198

Epoch

208/208 [==============================] - 1s 3ms/step - loss: 2786.4304 - val_loss: 1032.3684

Epoch 00003: val_loss improved from 3285.48205 to 1032.36842, saving model to arrPower_usage0_best_model_fold3_20_04_43.h5
Epoch 4/40
208/208 [==============================] - 1s 3ms/step - loss: 1072.8431 - val_loss: 520.6503

Epoch 00004: val_loss improved from 1032.36842 to 520.65026, saving model to arrPower_usage0_best_model_fold3_20_04_43.h5
Epoch 5/40
208/208 [==============================] - 1s 3ms/step - loss: 649.0638 - val_loss: 301.0414

Epoch 00005: val_loss improved from 520.65026 to 301.04135, saving model to arrPower_usage0_best_model_fold3_20_04_43.h5
Epoch 6/40
208/208 [==============================] - 1s 3ms/step - loss: 486.0723 - val_loss: 448.5011

Epoch 00006: val_loss did not improve from 301.04135
Epoch 7/40
208/208 [==============================] - 1s 3ms/step - loss: 399.4431 - val_loss: 351.0861

Epoch 00007: val_loss did not improve from 301.04135
Epoch 8/40


In [41]:
print ('start time: ', start_time)
print ('end time:', end_time)

start time:  2019-10-30 19:47:59.987027
end time: 2019-10-30 20:05:17.004893


In [12]:
# Plot and see data distribution

from matplotlib import pyplot as plt
import seaborn as sns

def plotAttrDataOfId(data, compute, features):
    plt.figure(figsize=(30, 20))
    for i, v in enumerate(features):
        plt.subplot(10, 3, i+1)
        cDf = df[df['compute']==compute]
        plt.plot(cDf['timespan'], cDf[v])
        plt.title(v)
        plt.tight_layout()

In [13]:
# for x in np.random.randint(0, len(computes), 3):
#     plotAttrDataOfId(df, computes[x], features)

In [14]:
def plotDataDistribution(data, features):
    plt.figure(figsize=(30, 10))
    for i, v in enumerate(features):
        plt.subplot(3, 10, i+1)
        sns.distplot(list(data[v].values))
        plt.title(v)
    plt.tight_layout()

In [ ]:
# plotDataDistribution(df, features)

In [15]:
from datetime import datetime
print(datetime.now().strftime("%H_%M_%S"))

17_54_25


In [ ]:
sns.distplot(y_train)

In [ ]:
sns.distplot(y_test)

In [ ]:
print (type(X_dfs[0]))